In [1]:
import io
import os

import pandas as pd
import numpy as np

import json

import requests

In [2]:
df = pd.read_csv("insta_data.csv")

In [61]:
key_file = open('google-vision-api-key.txt', 'r')
key = list(l for l in key_file)[0]
googlevision_url = 'https://vision.googleapis.com/v1/images:annotate?key='+key
headers = {"Content-type": "application/json",
            "Accept": "text/plain"}
labels = []
exceptions = 0
for url in df['urls']:
    data = json.dumps({"requests": [{"features": [{"type": "LABEL_DETECTION"}],
                        "image": {"source": {
                            "imageUri": url
                                    }
                                }
                            }]
                        })
    try:
        r = requests.post(googlevision_url,data,headers)
        res = r.json()
        label_annotations = (res['responses'][0]['labelAnnotations'])
        ls = []
        for l in label_annotations:
            ls.append(l['description'])
        labels.append(ls)
    except:
        exceptions += 1
        labels.append(None)

In [64]:
df['labels'] = labels

In [65]:
df.head()

,Unnamed: 0,urls,captions,comments,likes,labels
0,0,https://scontent-sea1-1.cdninstagram.com/vp/78...,Photo by @amivitale | Ntipiyon Nonguta and her...,278,61459,"[people, public space, community, tribe, tree,..."
1,1,https://scontent-sea1-1.cdninstagram.com/vp/53...,Video by @amivitale | Wild adult elephants dri...,362,83092,None
2,2,https://scontent-sea1-1.cdninstagram.com/vp/1e...,Photo by George Steinmetz @geosteinmetz | Epic...,431,136026,"[air sports, paragliding, landscape, sand, eco..."
3,3,https://scontent-sea1-1.cdninstagram.com/vp/a4...,Photo by @irablockphoto | A nighttime view fro...,1934,304484,"[waterway, city, cityscape, reflection, sky, l..."
4,4,https://scontent-sea1-1.cdninstagram.com/vp/d6...,Photo by Michael Yamashita @yamashitaphoto | H...,907,413142,"[mountain range, mountainous landforms, massif..."


In [66]:
df.to_csv('insta_data_with_labels.csv')